# reader

> Necessary scripts to read orbits from different formats

In [ ]:
#| default_exp reader

In [ ]:
#| export
import h5py
from scipy.io import loadmat
import os
import numpy as np
from pathlib import Path
import pandas as pd

In [ ]:
from unittest.mock import patch, MagicMock

## Loading Data

This function loads orbit data from .mat or .h5

In [ ]:
#| export
def load_orbit_data(file_path, variable_name=None, dataset_path=None):
    """
    Load orbit data from MATLAB .mat files or HDF5 .h5 files.

    Parameters:
    - file_path: str, the path to the .mat or .h5 file.
    - variable_name: str, optional, the name of the variable in the .mat file.
    - dataset_path: str, optional, the path to the dataset in the .h5 file.

    Returns:
    - data: The loaded orbit data.
    """
    if file_path.endswith('.mat'):
        if variable_name is None:
            raise ValueError("variable_name must be provided for .mat files")
        mat = loadmat(file_path)
        if variable_name in mat:
            data = mat[variable_name]
        else:
            raise ValueError(f"{variable_name} not found in {file_path}")
    
    elif file_path.endswith('.h5'):
        with h5py.File(file_path, 'r') as file:
            if dataset_path:
                if dataset_path in file:
                    data = np.array(file[dataset_path])
                else:
                    raise ValueError(f"{dataset_path} not found in {file_path}")
    
    else:
        raise ValueError("Unsupported file format. Please provide a .mat or .h5 file.")
    
    return data

In [ ]:
#| test "load_orbit_data with .mat file"
mock_mat_data = {'Xarray': np.array([1, 2, 3])}
mock_h5_data = np.array([4, 5, 6])

# Test for load_orbit_data with .mat file
with patch('__main__.loadmat', return_value=mock_mat_data) as mock_loadmat:
    result = load_orbit_data('test_data.mat', variable_name='Xarray')
    assert (result == mock_mat_data['Xarray']).all(), "MAT file loading failed or data mismatch"
    mock_loadmat.assert_called_once_with('test_data.mat')

# Test for load_orbit_data with .h5 file
with patch('__main__.h5py.File') as mock_h5py:
    mock_file = MagicMock()
    mock_file.__enter__.return_value = {'/files/PERIODIC ORBITS': mock_h5_data}
    mock_h5py.return_value = mock_file
    result = load_orbit_data('test_data.h5', dataset_path='/files/PERIODIC ORBITS')
    assert (result == mock_h5_data).all(), "H5 file loading failed or data mismatch"

### Functions to return data directly

In [ ]:
#| export
def get_orbit_features(data_directory=None):
    """
    Load orbit feature data from a MAT file located in the specified directory or a default 'Data' directory.

    :param data_directory: Optional; The directory where the MAT file is stored. If not specified, 
                           the function assumes a default directory relative to the script's location.
    :type data_directory: str, optional
    
    :return: A DataFrame containing detailed orbit features such as family, positions, velocities, 
             Jacobi constant, period, and stability index.
    :rtype: pandas.DataFrame
    """

    # Set default data directory if not provided
    if data_directory is None:
        # Get the directory of the current script
        script_directory = os.path.dirname(__file__)
        data_directory = os.path.join(script_directory, "Data")
    
    # Path to the MAT file containing orbit features
    feat_route = os.path.join(data_directory, "EM_IC_ARRAY.mat")
    
    # Load data from the MAT file
    feat_data = loadmat(feat_route)['out_EM']
    
    # Define column labels for the DataFrame
    column_labels = [
        'Orbit Family', 'Initial Position X', 'Initial Position Y', 'Initial Position Z',
        'Initial Velocity X', 'Initial Velocity Y', 'Initial Velocity Z',
        'Jacobi Constant', 'Period', 'Stability Index'
    ]
    
    # Create a DataFrame from the loaded data
    features = pd.DataFrame(feat_data, columns=column_labels)

    return features

In [ ]:
#| export
def get_orbit_data_and_labels(data_directory=None):
    """
    Load orbit data from an HDF5 file and corresponding labels from a MAT file.

    If no data_directory is specified, a default relative path is used. The function expects specific 
    file structures ('em_orbits.h5' and 'EM_IC_ARRAY.mat') within the directory.

    Returns:
        tuple: 
            - orbit_data (numpy.ndarray): Reshaped and transposed orbit data suitable for analysis.
            - labels (pandas.Series): Corresponding orbit classification labels, replicated as necessary.

    The function reshapes the orbit data and adjusts dimensions to prepare for machine learning or other analyses.
    Labels are extracted from a MAT file and matched to the reshaped orbit data.
    """

    # Define the default data directory based on the script's location if not provided
    if data_directory is None:
        data_directory = os.path.join(os.path.dirname(__file__), "Data")
    
    # Get Orbit Data
    hdf5_file_path = os.path.join(data_directory, "em_orbits.h5")
    with h5py.File(hdf5_file_path, 'r') as file:
        data = np.array(file['/files/PERIODIC ORBITS'])
    reshaped_array = data.reshape(36071, 7, 5, 1500)
    orbit_data = reshaped_array.transpose(0, 2, 1, 3).reshape(36071 * 5, 7, 1500)
    
    # Get Orbit Labels
    labels_df = get_orbit_features(orbit_directory)
    labels = pd.Series(labels_df['Orbit Family']).repeat(5).reset_index(drop=True)

    return orbit_data, labels


In [ ]:
def save_to_hdf5(data, file_name):
    """
    Save a numpy array to an HDF5 file.
    
    Args:
    data (numpy.ndarray): The data to save.
    file_name (str): The name of the HDF5 file.
    """
    # Open a new HDF5 file
    with h5py.File(file_name, 'w') as f:
        # Create a dataset in the file
        f.create_dataset('orbit_data', data=data, compression='gzip', compression_opts=9)

In [ ]:
#| export
def get_example_orbit_data():
    """
    Load orbit data from a hardcoded MAT file located in the `data` directory.
    
    The function is specifically designed to load the 'Xarray' variable 
    from the '1_L2_S_200_EM_CR3BP.mat' file. This setup is intended for 
    demonstration or testing purposes, where the data file and the variable 
    of interest are known ahead of time.

    :return: A numpy.ndarray containing the transposed data from the MAT file.
    """
    # Hardcoded file name and variable name
    filename = "1_L2_S_200_EM_CR3BP.mat"
    variable_name = 'Xarray'
    
    # Assuming the notebook or script is executed in a directory at the same level as the `data` folder
    current_dir = Path.cwd().parent
    matlab_file_path = current_dir / "data" / filename
    
    # Assuming `load_orbit_data` is a predefined function that loads and returns data from the .mat file
    data_mat = load_orbit_data(str(matlab_file_path), variable_name=variable_name)
    # Transpose the data for further use
    data_mat = np.transpose(data_mat, (2, 1, 0))
    
    return data_mat


In [ ]:
data = get_example_orbit_data()
data.shape

(200, 6, 300)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()